In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv")

In [ ]:
sub.head()

In [ ]:
test.head()

In [ ]:
train.head()

# 2. Preprocessing
### Score : ROC_CURVE

In [ ]:
allData = pd.concat([train, test])
allData

In [ ]:
categories = [col for col in allData.columns[1:] if allData[col].dtypes == 'object']
num_features = [col for col in allData.columns[1:] if allData[col].dtypes != 'object']

In [ ]:
le = LabelEncoder()
for col in allData.columns[allData.dtypes == 'object']:
    allData[col] = le.fit_transform(list(allData[col]))
allData.head()

In [ ]:
allData2 = allData.drop(['id', 'target'],1)
label = train['target']

In [ ]:
train2 = allData2[:len(train)]
test2 = allData2[len(train):]
train2.shape, test2.shape

# 3. Modeling : StratifyKFold and Ensemble

In [ ]:
# models with training

def model_catboost(X_train, y_train, X_valid, y_valid, test2):
    cat = CatBoostClassifier(task_type='GPU')
    cat.fit(X_train, y_train, verbose=200, cat_features = categories)
    valid_pred = cat.predict_proba(X_valid)[:,1]
    test_pred = cat.predict_proba(test2)[:,1]
    score_pred = roc_auc_score(y_valid, valid_pred)
    return score_pred, test_pred

def model_lgbm(X_train, y_train, X_valid, y_valid, test2):
    lgb = LGBMClassifier()
    lgb.fit(X_train, y_train)
    valid_pred = lgb.predict_proba(X_valid)[:,1]
    test_pred = lgb.predict_proba(test2)[:,1]
    score_pred = roc_auc_score(y_valid, valid_pred)
    return score_pred, test_pred

def model_xgb(X_train, y_train, X_valid, y_valid, test2):
    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)
    valid_pred = xgb.predict_proba(X_valid)[:,1]
    test_pred = xgb.predict_proba(test2)[:,1]
    score_pred = roc_auc_score(y_valid, valid_pred)
    return score_pred, test_pred

## 3-1. StratifyKFold

In [ ]:
%%time
# StratifyKFold

n_counts = 5

skf = StratifiedKFold(n_splits = n_counts, shuffle = True, random_state=42)

result = {}
scores = {}

model_list = ['cat', 'lgb', 'xgb']
for m in model_list:
    result[m] = 0
    scores[m] = []

i = 1
for train_index, valid_index in skf.split(train2, label):
    X_train, y_train = train2.loc[train_index], label[train_index]
    X_valid, y_valid = train2.loc[valid_index], label[valid_index]
    
    print(f"\n skf times # {i}\n")
    for m in model_list:
        if m == 'cat':
            score_pred, test_pred = model_catboost(X_train, y_train, X_valid, y_valid, test2)
        elif m == 'lgb':
            score_pred, test_pred = model_lgbm(X_train, y_train, X_valid, y_valid, test2)
        elif m == 'xgb':
            score_pred, test_pred = model_xgb(X_train, y_train, X_valid, y_valid, test2)
        scores[m].append(score_pred)
        result[m] += test_pred/n_counts 
        print(f"times # {i} {m} train and inference complete")
        
    i += 1
    
print('\n####################### DONE ###############################')

## 3-2. Ensemble

In [ ]:
for key, score in scores.items():
    print(f"{key} : {np.mean(score)}")

In [ ]:
weights = [0.3, 0.35, 0.35]
final_score = 0
for k, w in zip(result.keys(), weights):
    final_score += result[k] * w
    print(f"{k} result : {result[k]}")
print(f"final_score : {final_score}")

# 4. Submission

In [ ]:
sub['target'] = final_score
sub.to_csv("tabular_sub.csv", index=0)
sub